In [1]:
URL = "chals.sekai.team:3001"

import utils
import socket
import sys
import time


class CryptoClient:
    def __init__(self, connection: str):
        self.ip, self.port = connection.split(":")
        self.port = int(self.port)
        self.sock = None
        self.key = None
        self.buffer = ""  # Initialize an empty buffer

    def connect(self):
        self.sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        try:
            self.sock.connect((self.ip, self.port))
        except ConnectionError:
            print("Failed to connect to server.")
            sys.exit(1)
        print("CONNECTED")

    def read_line(self):
        while "\n" not in self.buffer:
            try:
                data = self.sock.recv(1024).decode('utf-8')
                if not data:
                    return None
                self.buffer += data
            except socket.error:
                return None
        line, self.buffer = self.buffer.split("\n", 1)
        return line

    def receive_key(self):
        line = self.read_line()
        if line:
            self.key = bytes.fromhex(line.split(':')[-1].strip())
        else:
            print("Failed to receive data from server.")
            sys.exit(1)
        print(f"KEY: {self.key.hex()}")

    def skip_until(self, text, debug=True):
        time.sleep(1)
        while True:
            line = self.read_line()
            if debug and line:
                print(line)
            if line and text in line:
                break

    def receive_query(self):
        line = self.read_line()
        if line:
            values = line.split(': ')[-1].split()
            query_from, query_to = map(int, values)
        else:
            print("Failed to receive data from server.")
            sys.exit(1)
        print(f"QUERY: {query_from} -> {query_to}")
        return query_from, query_to

    def receive_response(self):
        line = self.read_line()
        if line:
            encrypted_response = bytes.fromhex(line.split('Response: ')[-1].strip())
        else:
            print("Failed to receive data from server.")
            sys.exit(1)
        print(f"ENCRYPTED RESPONSE: {encrypted_response.hex()}")
        return encrypted_response
    
    def send_response(self, response: str):
        try:
            self.sock.sendall((response + "\n").encode('utf-8'))
        except socket.error:
            print("Failed to send data to server.")
            sys.exit(1)
        print(f"SENT RESPONSE: {response}")
    
SECURITY_PARAMETER = 16

def decrypt(u: int, v: int, key: bytes, resp: bytes) -> str:
    """
    Decrypts the response to reveal the shortest path between nodes in the graph.

    Parameters:
    u (int): The source node
    v (int): The destination node
    resp (bytes): The encrypted response from the server
    key (bytes): The secret key used for decryption

    Returns:
    str: The decrypted shortest path as a space-separated string
    """
    # Extract Subkeys
    key_SKE = key[:SECURITY_PARAMETER]
    key_DES = key[SECURITY_PARAMETER:]
    
    # Initialize Variables
    path = [u]
    # Decryption Loop
    segment_size = 32  # Assuming each encrypted node is 32 bytes
    for i in range(0, len(resp), segment_size):
        # Decrypt the segment
        block = resp[i:i+segment_size]
        decrypted_segment = utils.SymmetricDecrypt(key_SKE, block)
        
        # Path Extraction
        # Assuming decrypted_segment contains the node as bytes
        node1, node2 = map(int, decrypted_segment.decode().split(','))
        path.append(node1)
        
    # Validate the path starts at 'u' and ends at 'v'
    if path[0] != u or path[-1] != v:
        return "Invalid path"
    
    # Return Result
    return " ".join(map(str, path))


if __name__ == "__main__":
    client = CryptoClient(URL)
    client.connect()
    client.skip_until('Generating random graph', debug=False)
    client.receive_key()
    client.skip_until('Answer 50 queries', debug=False)
    
    for i in range(50):
        query_from, query_to = client.receive_query()
        encrypted_response = client.receive_response()
        decrypted_response = decrypt(query_from, query_to, client.key, encrypted_response)
        print(f"Decrypted response: {decrypted_response}")
        # Send the decrypted response back to the server
        client.send_response(decrypted_response)
    client.skip_until("Flag:")

CONNECTED
KEY: fac49a99920a6fd0175bffe083722e3ba606be19c8f4954ca6fb9293f2bdc2a0
QUERY: 68 -> 39
ENCRYPTED RESPONSE: 25acf6ceb0a810e7fa287199e7bbf40b4635fb40b33fe321fd4040c2e5a8bcb5faf52dffe5463009fafad5bb7ab90068feecf99f5db5779460458ebde4cd2224a31aa50197cfc5b7cc1dcf5883510d82192d2298b790766a36a3d6a80fbd125490b49adbdedd602c00bcb7a0789cd8567d3b62d0cb73e20c591c1cff95881be243d268d108de872b8b64184c83175f9293a9fa899d6491c108afc85a60ca2a88
Decrypted response: 68 25 78 116 76 39
SENT RESPONSE: 68 25 78 116 76 39
QUERY: 57 -> 74
ENCRYPTED RESPONSE: 38047e26e67dba9b30b6c60201037e949d41f897f155239023cc8b400ce629637bcffe5d60f27fb33645a07455a47936a906cafc911ba0f7ddbad68061d3078c
Decrypted response: 57 113 74
SENT RESPONSE: 57 113 74
QUERY: 20 -> 78
ENCRYPTED RESPONSE: db33d53b498bfb301d38a53347bb7efaaad00588375de37ddc09bf401b8c6702500ff3f3af4370cbb42d19d50efd53333275cb23a7f308a95913151b7c180ca437fcf3076c79cff4f0659ddabd644b9bcf691d6c5b482f4a3488d0132315ebf342f8931cd95f1db49662e9eaf57559cf408f227064